In [1]:
import numpy as np
import time as t
import os
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from matplotlib.ticker import FormatStrFormatter

In [2]:
# initial
R_max = 1
eps = 1 # diffusion coefficient
n = 9
# for time
final_time = 80
t_step = 0.005
Max_step = int(final_time/t_step) # calculate the number of time steps
time = np.arange(0,final_time+t_step,t_step) 
time = time[1:]
ratio = 4
# time for draw
time1 = time[::ratio] # 切片（slicing）语法，提取time数组中的元素，提出每隔ratio个元素的子集
Rstep1 = time[1] - time[0]
Max_step1 = len(time1)
# for sptial
N = 91
delta_r = (2*R_max)/(2*(N-1)+1)
r = np.arange(1, N+1, dtype=np.float64) # 生成左闭右开的范围，步长默认为1
r = (r- 1/2)*delta_r

lamda = 40*r**4+30*1j*r**2
uu = np.zeros((N,Max_step1),dtype=np.complex128)
# uu = np.zeros((N,Max_step1))

In [3]:
def getKcomplex(r, n):
    M = len(r)
    x, y = np.meshgrid(r, r)
    if n==0:
        kernel = (y * ((-2 / 3) * x ** 4 + (35 / 96) * x ** 6 + (1j * (-65 / 504)) * x ** 8 + ((-11 / 945) + 1j * (155 / 10752)) * x ** 10 +
                                        (157 / 24192) * x ** 12 + ((-635 / 2408448) + 1j * (-179 / 174636)) * x ** 14 + (-8 / 3) * x ** 2 * y ** 2 +
                                        (105 / 32) * x ** 4 * y ** 2 + (1j * (-130 / 63)) * x ** 6 * y ** 2 + ((-55 / 189) + 1j * (3875 / 10752)) * x ** 8 * y ** 2 +
                                        (157 / 672) * x ** 10 * y ** 2 + ((-635 / 49152) + 1j * (-179 / 3564)) * x ** 12 * y ** 2 + (-2 / 3) * y ** 4 +
                                        (-45 / 32) * x ** 2 * y ** 4 + (1j * (-95 / 28)) * x ** 4 * y ** 4 + ((-220 / 189) + 1j * (4075 / 5376)) * x ** 6 * y ** 4 +
                                        (3275 / 2688) * x ** 8 * y ** 4 + ((-10235 / 114688) + 1j * (-265 / 616)) * x ** 10 * y ** 4 + (-215 / 96) * y ** 6 +
                                        (1j * (220 / 63)) * x ** 2 * y ** 6 + ((-80 / 189) + 1j * (-10625 / 5376)) * x ** 4 * y ** 6 + (125 / 3024) * x ** 6 * y ** 6 +
                                        ((54125 / 1032192) + 1j * (-40375 / 49896)) * x ** 8 * y ** 6 + (1j * (1055 / 504)) * y ** 8 +
                                        ((295 / 189) + 1j * (1775 / 10752)) * x ** 2 * y ** 8 + (-9725 / 2688) * x ** 4 * y ** 8 +
                                        ((60125 / 147456) + 1j * (6400 / 6237)) * x ** 6 * y ** 8 + ((311 / 945) + 1j * (7295 / 10752)) * y ** 10 +
                                        (33 / 32) * x ** 2 * y ** 10 + ((-51165 / 114688) + 1j * (367 / 198)) * x ** 4 * y ** 10 + (26293 / 24192) * y ** 12 +
                                        ((-15665 / 1032192) + 1j * (-54349 / 49896)) * x ** 2 * y ** 12 + ((748105 / 7225344) + 1j * (-174565 / 349272)) * y ** 14 +
                                        0.5 * ((1j * (-5)) * x ** 2 + (1j * (-5)) * y ** 2)))
    elif n in [1, -1]:
        kernel= (x ** (-1) * y ** 2 * ((1j * (-5)) * x ** 2 + (-2) * x ** 4 + (5 / 4) * x ** 6 + (1j * (-25 / 42)) * x ** 8 +
                                       ((-17 / 252) + 1j * (25 / 336)) * x ** 10 + (1 / 24) * x ** 12 + ((-5 / 2688) + 1j * (-65 / 8316)) * x ** 14 +
                                       (-2) * x ** 2 * y ** 2 + (15 / 4) * x ** 4 * y ** 2 + (1j * (-25 / 7)) * x ** 6 * y ** 2 +
                                       ((-85 / 126) + 1j * (125 / 168)) * x ** 8 * y ** 2 + (5 / 8) * x ** 10 * y ** 2 +
                                       ((-5 / 128) + 1j * (-65 / 396)) * x ** 12 * y ** 2 + (-5) * x ** 2 * y ** 4 +
                                       (1j * (-15 / 14)) * x ** 4 * y ** 4 + ((-85 / 63) + 1j * (-25 / 336)) * x ** 6 * y ** 4 +
                                       (75 / 56) * x ** 8 * y ** 4 + ((-275 / 2688) + 1j * (-8165 / 11088)) * x ** 10 * y ** 4 +
                                       (1j * (110 / 21)) * x ** 2 * y ** 6 + ((125 / 126) + 1j * (-50 / 21)) * x ** 4 * y ** 6 +
                                       (-325 / 168) * x ** 6 * y ** 6 + ((2375 / 8064) + 1j * (-2575 / 8316)) * x ** 8 * y ** 6 +
                                       ((277 / 252) + 1j * (275 / 168)) * x ** 2 * y ** 8 + (-20 / 7) * x ** 4 * y ** 8 +
                                       ((25 / 168) + 1j * (11675 / 5544)) * x ** 6 * y ** 8 + (39 / 14) * x ** 2 * y ** 10 +
                                       ((-95 / 168) + 1j * (35 / 66)) * x ** 4 * y ** 10 + ((535 / 2016) + 1j * (-15725 / 11088)) * x ** 2 * y ** 12))
    elif n in [2, -2]:
        kernel = (x ** (-2) * y ** 3 * ((-4) * x ** 4 + (85 / 32) * x ** 6 + (1j * (-45 / 28)) * x ** 8 + ((-2 / 9) + 1j * (2285 / 10752)) * x ** 10 +
                                       (65 / 448) * x ** 12 + ((-16435 / 2408448) + 1j * (-89 / 2772)) * x ** 14 + (85 / 32) * x ** 4 * y ** 2 +
                                       (1j * (-30 / 7)) * x ** 6 * y ** 2 + ((-10 / 9) + 1j * (11425 / 10752)) * x ** 8 * y ** 2 +
                                       (65 / 56) * x ** 10 * y ** 2 + ((-82175 / 1032192) + 1j * (-445 / 1188)) * x ** 12 * y ** 2 +
                                       (-225 / 32) * x ** 2 * y ** 4 + (1j * (15 / 7)) * x ** 4 * y ** 4 + ((-10 / 9) + 1j * (-7675 / 5376)) * x ** 6 * y ** 4 +
                                       (75 / 112) * x ** 8 * y ** 4 + ((-13625 / 344064) + 1j * (-725 / 792)) * x ** 10 * y ** 4 +
                                       (55 / 32) * y ** 6 + (1j * 5) * x ** 2 * y ** 6 + ((22 / 9) + 1j * (-8675 / 5376)) * x ** 4 * y ** 6 +
                                       (-1195 / 336) * x ** 6 * y ** 6 + ((169175 / 344064) + 1j * (1315 / 1848)) * x ** 8 * y ** 6 +
                                       (1j * (-5 / 4)) * y ** 8 + (1j * (1125 / 512)) * x ** 2 * y ** 8 + (-395 / 448) * x ** 4 * y ** 8 +
                                       ((-291325 / 1032192) + 1j * (365 / 154)) * x ** 6 * y ** 8 + (1j * (-1545 / 3584)) * y ** 10 +
                                       (345 / 112) * x ** 2 * y ** 10 + ((-18185 / 49152) + 1j * (-431 / 462)) * x ** 4 * y ** 10 +
                                       (-415 / 672) * y ** 12 + ((5625 / 16384) + 1j * (-25 / 24)) * x ** 2 * y ** 12 +
                                       ((-45575 / 802816) + 1j * (325 / 1512)) * y ** 14 + 0.5 * ((1j * (-15)) * x ** 2 + (1j * 5) * y ** 2)))
    elif n in [3, -3]:
        kernel = (x ** (-3) * y ** 4 * ((-20 / 3) * x ** 4 + (55 / 12) * x ** 6 + (1j * (-425 / 126)) * x ** 8 +
                                       ((-209 / 378) + 1j * (155 / 336)) * x ** 10 + (565 / 1512) * x ** 12 +
                                       ((-1025 / 56448) + 1j * (-33505 / 349272)) * x ** 14 +
                                       (10 / 3) * x ** 2 * y ** 2 + (1j * (-425 / 126)) * x ** 6 * y ** 2 +
                                       ((-1045 / 756) + 1j * (775 / 672)) * x ** 8 * y ** 2 + (565 / 336) * x ** 10 * y ** 2 +
                                       ((-1025 / 8064) + 1j * (-33505 / 49896)) * x ** 12 * y ** 2 +
                                       (-2 / 3) * y ** 4 + (-15 / 2) * x ** 2 * y ** 4 + (1j * 5) * x ** 4 * y ** 4 +
                                       ((-209 / 378) + 1j * (-125 / 42)) * x ** 6 * y ** 4 + (-95 / 112) * x ** 8 * y ** 4 +
                                       ((15 / 128) + 1j * (-2201 / 2772)) * x ** 10 * y ** 4 +
                                       (35 / 12) * y ** 6 + (1j * (65 / 18)) * x ** 2 * y ** 6 + (100 / 27) * x ** 4 * y ** 6 +
                                       (-1565 / 378) * x ** 6 * y ** 6 + ((275 / 504) + 1j * (200455 / 99792)) * x ** 8 * y ** 6 +
                                       (1j * (-235 / 126)) * y ** 8 + ((-275 / 189) + 1j * (225 / 112)) * x ** 2 * y ** 8 +
                                       (75 / 56) * x ** 4 * y ** 8 + ((-725 / 1008) + 1j * (81815 / 49896)) * x ** 6 * y ** 8 +
                                       ((181 / 756) + 1j * (-145 / 224)) * y ** 10 +
                                       (835 / 336) * x ** 2 * y ** 10 + (1j * (-35 / 18)) * x ** 4 * y ** 10 + (-385 / 432) * y ** 12 +
                                       ((125 / 448) + 1j * (-895 / 2268)) * x ** 2 * y ** 12 +
                                       ((-485 / 6272) + 1j * (16007 / 63504)) * y ** 14 +
                                       0.5 * ((1j * (-20)) * x ** 2 + (1j * 10) * y ** 2)))
    elif n in [4, -4]:
        kernel = (x ** (-4) * y ** 5 * ((-10) * x ** 4 + (225 / 32) * x ** 6 + (1j * (-1025 / 168)) * x ** 8 +
                                        ((-73 / 63) + 1j * (9175 / 10752)) * x ** 10 + (2155 / 2688) * x ** 12 +
                                        ((-287825 / 7225344) + 1j * (-13675 / 58212)) * x ** 14 +
                                        8 * x ** 2 * y ** 2 + (-135 / 32) * x ** 4 * y ** 2 +
                                        ((-73 / 63) + 1j * (9175 / 10752)) * x ** 8 * y ** 2 + (431 / 224) * x ** 10 * y ** 2 +
                                        ((-57565 / 344064) + 1j * (-2735 / 2772)) * x ** 12 * y ** 2 +
                                        (-2) * y ** 4 + (-205 / 32) * x ** 2 * y ** 4 + (1j * (25 / 4)) * x ** 4 * y ** 4 +
                                        (1j * (-1125 / 256)) * x ** 6 * y ** 4 + (-8095 / 2688) * x ** 8 * y ** 4 +
                                        ((125935 / 344064) + 1j * (-485 / 1848)) * x ** 10 * y ** 4 +
                                        (115 / 32) * y ** 6 + (1j * (40 / 21)) * x ** 2 * y ** 6 +
                                        ((100 / 21) + 1j * (3825 / 1792)) * x ** 4 * y ** 6 + (-375 / 112) * x ** 6 * y ** 6 +
                                        ((404525 / 1032192) + 1j * (18055 / 5544)) * x ** 8 * y ** 6 +
                                        (1j * (-115 / 56)) * y ** 8 + ((-65 / 21) + 1j * (4425 / 3584)) * x ** 2 * y ** 8 +
                                        (2775 / 896) * x ** 4 * y ** 8 + (-16875 / 16384) * x ** 6 * y ** 8 +
                                        ((41 / 63) + 1j * (-7325 / 10752)) * y ** 10 +
                                        (985 / 672) * x ** 2 * y ** 10 +
                                        ((48675 / 114688) + 1j * (-275 / 126)) * x ** 4 * y ** 10 +
                                        (-2497 / 2688) * y ** 12 +
                                        ((2065 / 16384) + 1j * (325 / 1512)) * x ** 2 * y ** 12 +
                                        ((-57215 / 802816) + 1j * (7565 / 38808)) * y ** 14 +
                                        0.5 * ((1j * (-25)) * x ** 2 + (1j * 15) * y ** 2)))
    elif n in [5, -5]:
        kernel = (x ** (-5) * y ** 6 * ((-14) * x ** 4 + 10 * x ** 6 + (1j * (-10)) * x ** 8 +
                                        ((-389 / 180) + 1j * (955 / 672)) * x ** 10 + (73 / 48) * x ** 12 +
                                        ((-1445 / 18816) + 1j * (-309 / 616)) * x ** 14 +
                                        14 * x ** 2 * y ** 2 + (-10) * x ** 4 * y ** 2 + (1j * (20 / 3)) * x ** 6 * y ** 2 +
                                        (73 / 48) * x ** 10 * y ** 2 + ((-1445 / 8064) + 1j * (-103 / 88)) * x ** 12 * y ** 2 +
                                        (-4) * y ** 4 + (-15 / 4) * x ** 2 * y ** 4 + (1j * (65 / 14)) * x ** 4 * y ** 4 +
                                        (1j * (-75 / 14)) * x ** 6 * y ** 4 + (-75 / 14) * x ** 8 * y ** 4 +
                                        ((115 / 168) + 1j * (2425 / 3696)) * x ** 10 * y ** 4 +
                                        (15 / 4) * y ** 6 + (1j * (5 / 7)) * x ** 2 * y ** 6 +
                                        ((35 / 6) + 1j * (125 / 28)) * x ** 4 * y ** 6 + (-25 / 21) * x ** 6 * y ** 6 +
                                        (1j * (25 / 6)) * x ** 8 * y ** 6 + (1j * (-85 / 42)) * y ** 8 +
                                        ((-175 / 36) + 1j * (25 / 672)) * x ** 2 * y ** 8 + (1325 / 336) * x ** 4 * y ** 8 +
                                        ((-125 / 112) + 1j * (-125 / 54)) * x ** 6 * y ** 8 +
                                        ((107 / 90) + 1j * (-95 / 168)) * y ** 10 +
                                        (43 / 112) * x ** 2 * y ** 10 + ((45 / 56) + 1j * (-769 / 504)) * x ** 4 * y ** 10 +
                                        (-23 / 28) * y ** 12 + ((-185 / 2688) + 1j * (6463 / 11088)) * x ** 2 * y ** 12 +
                                        ((-2665 / 56448) + 1j * (445 / 4158)) * y ** 14 +
                                        0.5 * ((1j * (-30)) * x ** 2 + (1j * 20) * y ** 2)))
    elif n in [6, -6]:
        kernel = (x ** (-6) * y ** 7 * ((-56 / 3) * x ** 4 + (1295 / 96) * x ** 6 + (1j * (-275 / 18)) * x ** 8 +
                                        ((-100 / 27) + 1j * (1125 / 512)) * x ** 10 + (2281 / 864) * x ** 12 +
                                        ((-139295 / 1032192) + 1j * (-12085 / 12474)) * x ** 14 +
                                        (64 / 3) * x ** 2 * y ** 2 + (-555 / 32) * x ** 4 * y ** 2 + (1j * (1100 / 63)) * x ** 6 * y ** 2 +
                                        ((500 / 189) + 1j * (-5625 / 3584)) * x ** 8 * y ** 2 +
                                        ((-139295 / 1032192) + 1j * (-12085 / 12474)) * x ** 12 * y ** 2 +
                                        (-20 / 3) * y ** 4 + (15 / 32) * x ** 2 * y ** 4 + (1j * (-15 / 14)) * x ** 4 * y ** 4 +
                                        ((-250 / 189) + 1j * (-9925 / 1792)) * x ** 6 * y ** 4 + (-1375 / 192) * x ** 8 * y ** 4 +
                                        ((16875 / 16384) + 1j * (125 / 72)) * x ** 10 * y ** 4 +
                                        (325 / 96) * y ** 6 + (1j * (55 / 63)) * x ** 2 * y ** 6 +
                                        ((1390 / 189) + 1j * (35825 / 5376)) * x ** 4 * y ** 6 + (5975 / 3024) * x ** 6 * y ** 6 +
                                        ((-71875 / 114688) + 1j * (20875 / 4536)) * x ** 8 * y ** 6 +
                                        (1j * (-125 / 63)) * y ** 8 + ((-1286 / 189) + 1j * (-15275 / 10752)) * x ** 2 * y ** 8 +
                                        (155 / 42) * x ** 4 * y ** 8 + ((-104725 / 114688) + 1j * (-11425 / 2268)) * x ** 6 * y ** 8 +
                                        ((346 / 189) + 1j * (-3575 / 10752)) * y ** 10 +
                                        (-169 / 336) * x ** 2 * y ** 10 + ((121355 / 114688) + 1j * (5 / 308)) * x ** 4 * y ** 10 +
                                        (-7765 / 12096) * y ** 12 + ((-274105 / 1032192) + 1j * (4175 / 7128)) * x ** 2 * y ** 12 +
                                        ((-13225 / 1032192) + 1j * (1745 / 49896)) * y ** 14 +
                                        0.5 * ((1j * (-35)) * x ** 2 + (1j * 25) * y ** 2)))
    elif n in [7, -7]:
        kernel = (x ** (-7) * y ** 8 * ((-24) * x ** 4 + (35 / 2) * x ** 6 + (1j * (-155 / 7)) * x ** 8 +
                                        ((-125 / 21) + 1j * (45 / 14)) * x ** 10 + (30 / 7) * x ** 12 +
                                        ((-12485 / 56448) + 1j * (-33623 / 19404)) * x ** 14 +
                                        30 * x ** 2 * y ** 2 + (-105 / 4) * x ** 4 * y ** 2 + (1j * (465 / 14)) * x ** 6 * y ** 2 +
                                        ((625 / 84) + 1j * (-225 / 56)) * x ** 8 * y ** 2 + (-45 / 14) * x ** 10 * y ** 2 +
                                        (-10) * y ** 4 + (25 / 4) * x ** 2 * y ** 4 + (1j * (-85 / 7)) * x ** 4 * y ** 4 +
                                        ((-625 / 126) + 1j * (-775 / 168)) * x ** 6 * y ** 4 + (-625 / 84) * x ** 8 * y ** 4 +
                                        ((75 / 56) + 1j * (625 / 252)) * x ** 10 * y ** 4 +
                                        (5 / 2) * y ** 6 + (1j * (45 / 14)) * x ** 2 * y ** 6 +
                                        ((629 / 63) + 1j * (2825 / 336)) * x ** 4 * y ** 6 + (305 / 56) * x ** 6 * y ** 6 +
                                        ((-325 / 224) + 1j * (14195 / 3024)) * x ** 8 * y ** 6 +
                                        (1j * (-15 / 7)) * y ** 8 + ((-1145 / 126) + 1j * (-125 / 42)) * x ** 2 * y ** 8 +
                                        (135 / 56) * x ** 4 * y ** 8 + ((-25 / 64) + 1j * (-11065 / 1386)) * x ** 6 * y ** 8 +
                                        ((649 / 252) + 1j * (-5 / 336)) * y ** 10 +
                                        (-25 / 24) * x ** 2 * y ** 10 + ((3035 / 2688) + 1j * (1879 / 792)) * x ** 4 * y ** 10 +
                                        (-25 / 56) * y ** 12 + ((-3475 / 8064) + 1j * (655 / 4158)) * x ** 2 * y ** 12 +
                                        ((475 / 18816) + 1j * (925 / 77616)) * y ** 14 +
                                        0.5 * ((1j * (-40)) * x ** 2 + (1j * 30) * y ** 2)))
    else:
        kernel = np.zeros((M, M), dtype=np.complex128)
    
    # 建立索引
    mask = x - y < 0
    # 按索引赋值，索引为mask中的元素（即下三角区域）赋值为nan
    kernel[mask] = np.nan
    # 取最后一列，并转置
    k_bud = kernel[:,-1].T
    return k_bud, kernel



<details>
  <summary style="margin-top: 20px;"> 1. 确定积分区间和子区间数量</summary>

  设要求的定积分是：

  $$
  \int_a^b f(x) \, dx
  $$

  首先确定积分区间 \([a, b]\)。然后选择一个合适的偶数 \(n\)，将区间 \([a, b]\) 划分为 \(n\) 个子区间，每个子区间的宽度为：

  $$
  h = \frac{b - a}{n}
  $$

</details>

<details>
  <summary style="margin-top: 20px;"> 2. 计算节点的函数值</summary>

  计算子区间端点和中点处的函数值。节点：

  $$
  x_i = a + i h, \quad i = 0, 1, \cdots, n
  $$

  对于复合辛普森法则，需要计算 \(f(x_i)\) 的值，同时对于每个子区间 \([x_i, x_{i+1}]\)，还需要计算中点：

  $$
  x_{i+\frac{1}{2}} = x_i + \frac{h}{2}
  $$

  处的函数值 \(f(x_{i+\frac{1}{2}})\)。

</details>

<details>
  <summary style="margin-top: 20px;"> 3. 应用复合辛普森公式计算定积分近似值</summary>

  复合辛普森公式为：

  $$
  \int_a^b f(x) \, dx \approx \frac{h}{3} \left[ f(x_0) + f(x_n) + 4 \sum_{i=1, \text{odd}}^{n-1} f(x_i) + 2 \sum_{i=2, \text{even}}^{n-2} f(x_i) \right]
  $$

  其中：

  $$
  h = \frac{b - a}{n}
  $$

  按照公式中的系数，将前面计算得到的函数值进行组合计算，得到定积分的近似值。

</details>





In [4]:
# 生成用于求解复合辛普森积分的系数序列
def ComSimpson(N, hr):
    # hr = delta_r
    c = np.ones((1,N))*2
    # 在matlab中 c(2:2:N-1)=4
    # 比如 C[行索引, 1:n:2] 就会涉及到对特定行中按照列维度进行这样的切片选取并赋值操作了，和这里介绍的一维数组切片赋值在维度上有差异。
    # 下面的0，表示选取第一个维度的所有元素, 1:n:2 表示选取第二个维度的1到n-1的元素，步长为2
    c[0,1:N-1:2] = 4
    c[0,0] = 1
    c[0,N-1] = 1
    c = hr/3*c
    return c